In [1]:
import pandas as pd
import numpy as np
from scipy.stats import weibull_min
import csv


# load general admin building data
gen_admin_params = pd.read_csv('selected_15_components_data.csv')

In [2]:
# helper function for generating dynamics data

num_states = 101
num_actions = 1
states = np.arange(0, num_states, 1)
actions = np.arange(0, num_actions, 1)

def synthesize_dynamics(state, dynamics_shape, dynamics_scale):
    """
    Synthesize transition dynamics from the given state in the absence of maintenance action
    """
    probs = np.zeros(num_states)
    for next_state in range(state,-1,-1):
        probs[next_state] = weibull_min.pdf(101-next_state+1, dynamics_shape , scale=dynamics_scale)
    probs = probs/np.sum(probs)
    return probs

def gen_trans_prob(dynamics_shape, dynamics_scale):
    """
    A function that generates transition probability for states of the component
    """

    trans_prob = np.zeros((num_states, num_actions, num_states))
    for state in states:
        for action in range(num_actions):
            trans_prob[state, action, :] = synthesize_dynamics(state, dynamics_shape, dynamics_scale)    
    return trans_prob

In [4]:
# generate dynamics for each component

component_ids = list(gen_admin_params['component_id'])
dynamics_shapes = list(gen_admin_params['shape'])
dynamics_scales = list(gen_admin_params['scale'])

for i in range(len(component_ids)):
    trans_probs = gen_trans_prob(dynamics_shapes[i], dynamics_scales[i])

    # save to numpy array
    np.save(f'./dynamics/dynamics_{component_ids[i]}.npy', trans_probs)
    
    trans_probs = trans_probs.tolist()
    dynamics = open(f'./dynamics/dynamics_{component_ids[i]}.csv', 'w')
    writer = csv.writer(dynamics, quoting=csv.QUOTE_NONE, escapechar=' ')
    for data in trans_probs:
        writer.writerow(data)
    dynamics.close()